# Random Forest 

### 알고리즘은 강의 자료를 참고하겠음. 

1. For b =1 to B : 
- Draw a bootstrap sample $Z^*$ of size N from the training data. 
- Grow a random-forest tree $T_b$ to the bootstrapped data, by recursively repeating the following steps for each ternimal node of the tree, until the mininum node size $n_{min}$ is reached. 

> 1). Select m variables at random from the p variables 

> 2). Pick the best variable/split-point among the m 

> 3). Split the node into two daughter nodes 

2. Output the ensemble of trees {$T_b$}$|_1^B$ 
- To make a prediction at a new point x 
- Regression : $\hat f_{rf}^B(x) = \frac{1}{B} \sum_{b=1}^B T_b(x)$ 
- Classification : Let $\hat C_b(x)$ be the class prediction of the bth random-forest tree. Then $\hat C_{rf}^B(x)$ = majority vote{$\hat C_b(x)$}$|_1^B$



**구현해야하는 것**
- 사이즈 N인 Bootstrap 샘플 B개 뽑기 
- 샘플 데이터 셋 B개에 대해서 Random-forest tree 만들기 
- result aggregation 진행. Regression이면 평균 값을, Classification이면 Majority vote 적용 

**필요한 것**
- M : 랜덤 트리 개수  
- m : 선정할 변수 개수
- X : input 데이터 
- y : Output 데이터 
- Treenode 
- Split variable 및 기준 설정 

**함수의 형태** 
- def __init__(self, X, y, num_tree, num_var) 
- def make_tree(self, num_tree, num_var): 

*Regression 간에는 SVR을, classification 간에는 SVC를 적용하겠음*


In [8]:
import numpy as np
import pandas as pd
import random as rand

from sklearn.datasets import load_iris
X = load_iris()['data']
y = load_iris()["target"].reshape(-1,1)
D = np.concatenate((X,y), axis=1)
A = load_iris()['feature_names']

import matplotlib.pyplot as plt
import scipy as sc
from scipy.stats import norm
from sys import maxsize
from sklearn import svm


### Decision Tree 수도 알고리즘 
출처 - 단단한 머신러닝 챕터 4

입력 : 훈련세트 D = {(x1,y1), (x2,y2), ..., (xm,ym)}, 속성집합 A = {a1, a2, ..., ad} 
과정 : 함수 TreeGenerate(D,A) 

1. node 생성 
2. if D의 샘플이 모두 같은 클래스(y값)에 속하면 then 
- 해당 node를 레이블이 C인 터미널 노드로 정한다. return 
- end if 

3. (Leaf 설정) if A가 없거나, D의 샘플이 A 속성에 같은 값을 취한다면 then 
- 해당 node를 터미널 노드로 정하고, 해당 클래스는 D 샘플 중 가장 많은 샘풀의 수가 속한 속성으로 정한다. return 
- end if 

4. A에서 최적의 분할 속성 $a_*$를 선택한다. 

5. for $a_*$의 각 $a_*^v$에 대해 다음을 행한다. 
- node에서 하나의 가지를 생선한다. $D_v$는 D는 $a_*^v$ 속성값을 가지는 샘플의 하위 집합으로 표기한다. 
- if $D_v$가 0이면 then 
> 해당 가지 node를 터미널 노드로 정하고, 해당 클래스는 D 샘플 중 가장 많은 클래스로 정한다. return 
- else 
> TreeGenerate($D_v$, A/{$a_*$} 를 가지 노드로 정한다. 
> end if 
- end for

출력 : node를 루트 노드로 하는 의사결정 트리 



### 4. 최적의 분할 속성 정하는 방법 : 정보 이득율 사용 
- Gain_ratio(D,a) = $\frac{Gain(D,a)}{IV(a)}$ 
- IV(a) = - $\sum_{v=1}^V \frac{D^v}{D} log2 \frac{D^v}{D}$
- Gain(D,a) = Ent(D) - $\sum_{v=1}^V \frac{|D^v|}{|D|} Ent(D^v)$
- Ent(D) = -$\sum_{k=1}^{|Y|}p_k log2(p_k) $
> |Y| : 클래스의 종류 개수 
> $p_k$ : k번째 클래스의 비율 



### 5. for $a_*$의 각 value에 따라 for 절을 수행할 것. 문제 - Category value가 아니라 Numerical Value로 주어지면 어떻게 분류하지? 


In [44]:
#엔트로피 설정

from math import log 
from collections import defaultdict

def entropy(dataset) : 
    datasize = len(dataset) 
    
    label_count = defaultdict(int)
    entropy = 0.0 
    
    for data_line in dataset : 
        cur_label = data_line[-1]
        label_count[cur_label] += 1 
        
    for key in label_count : 
        prop = float(label_count[key]) / datasize 
        entropy -= prop*log(prop,2) 
    return entropy

# Dv, Ent(Dv) 계산하기 위한 토대. 특정 속성값을 제외한 나머지 값들을 결과로 반환 
def pick(dataset, index, value) : 
    result = [] 
    for dataline in dataset : 
        dataline = list(dataline)
        if dataline[index] == value : 
            temp_vec = dataline[:index]
            temp_vec.extend(dataline[index+1:])
            result.append(temp_vec)
    return result 

# 최고의 선별 속성을 결정하는 함수 
def best_feat(dataset): 
    base_entropy = entropy(dataset)
    best_infogain = 0.0 
    index = 0 
    num_feat = len(dataset[0]) -1 
    for i in range(num_feat) : 
        feat_value_set = set([value[i] for value in dataset])
        new_entropy = 0.0 
        
        # 정보 이득 계산하기 
        for element in feat_value_set : 
            sub_dataset = pick(dataset, i, element) 
            prop = len(sub_dataset) / float(len(dataset))
            new_entropy += prop*entropy(sub_dataset)
            
        info_gain = base_entropy - new_entropy 
        
        if best_infogain < info_gain : 
            best_infogain = info_gain 
            index = i 
    return index 

In [45]:
best_feat(D)

2

In [51]:
# 특정 속성 값의 개수에 맞춰 Child 생성하는데 막힘. 

class treenode() : 
    def __init__(self, num_child=None) : 
        self.label = None
        self.child = self.make_child(num_child)
        
    def make_child(self, num_child) : 
        for i in range(num_child) : 
            globals()['self.child_{}'.format(i)] = None
            
                

In [ ]:
def RandomTreeGenerate(D,A,m) : 
    node = treenode()
    num_att = len(D[0, :-1])
    chosen_att = np.choice(range(num_att), m)
    
    # "모든 샘플이 같은 클래스에 속하면"을 y값에 대한 분산값이 0인 것으로 확인 
    if len(D) != 0 and  np.var(D[:, -1]) ==0 : 
        node.label = D[0,-1] 
        return node
    
    # "속성 집합이 0일 경우"를  샘플의 개수로, 속성 값이 전부 같은 경우는 x값에 대한 분산값이 0인 것으로 확인 
    if len(D) == 0 or np.var(D[:, :-1]) == 0 : 
        # np.unique를 통해서 lable의 개수 세기. [1]은 갯수 list로, 그중 가장 큰 값을 가진 것의 index 반환  
        node.label = np.argmax(np.unique(D[:, -1], return_counts=True)[1])
        return node 
    

    best_att = best_feat(D)
"""
# 5번쨰 항목. Numerical Value의 이산화 작업 : 이분법 적용 
# 가장 가까운 split 포인트로 값을 재할당 
    value_list = np.uniqu(np.sort(D[:, best_att]))
    split_point = [] 
    pre = value_list[0]
    for i in range(len(value_list)-1) :
        split_point.append((pre+value_list[i+1])/2) 
        pre = value_list[i+1]
    
    for value in dataset[:, best_att] : 
        index = np.argmin(split_point - value)
"""

    for value in dataset[:,best_att] : 
        new_node = treenode()
        node.child = new_node
        new_dataset = [dataset[i] for i in len(D) if dataset[i,best_att] == value]
        if len(new_dataset) == 0 : 
            new_node.label = np.argmax(np.unique(D[:, -1], return_counts=True)[1])
            return new_node
        
    
    
    
        

In [6]:
class Random_forest(): 
    def __init__(self, data, num_sample, num_tree, num_var) : 
        self.data = data 
        self.X = data[:, :-1]
        self.y = data[:. -1]
        self.n = np.shape(self.X)[0]
        self.m = np.shape(self.X)[1]
        
        self.num_sample = num_sample
        self.num_tree = num_tree
        
        self.sample = self.random_sample() 
        
        self.num_var = num_var 
        
    def random_sample(self) : 
        sample_list = []
    
        for i in range(self.num_tree) : 
            np.random.seed(i)
            sample_index = np.random.choice(self.n, size = self.num_sample, replace = True)
            sample = X1[sample_index]
            sample_list.append(sample)

        return sample_list

        
    def make_tree(self) : 
        
        


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [ ]:
# 이진 분류 모델 
def bagging(X, x_new, model, T, N, kernel, agg_type) : 
    sample_list = []
    ensemble_list = [] 
    X1 = np.array([X[i][:-1] for i in range(len(X)) if X[i][-1] == 0]) 
    X2 = np.array([X[i][:-1] for i in range(len(X)) if X[i][-1] == 1]) 
    
    for i in range(T) : 
        np.random.seed(i)
        sample_index = np.random.choice(np.shape(X1)[0], size = N, replace = True)
        sample_1 = X1[sample_index]
        sample_2 = X2[sample_index]
        ensemble_list.append(model(sample_1,sample_2,kernel)) 
    
    result = [] 
    for i in range(T) : 
        result.append(ensemble_list[i].classify(x_new)) 
    
    return aggregation(result, agg_type)


In [ ]:
def aggregation(result, agg_type) : 
    
    if agg_type == "Majority" : 
        agg_result = argmax([result.count(0), result.count(1)]) 
        return agg_result
    
    elif agg_type == "Accuracy Weighted" : 
        return 0
        # 모델별 정확도가 구현되어 있지 않아 여기선 생략 
    
    elif agg_type == "Prediction Weighted" : 
        return 0 
        # 모델별 예상 확률이 구현되어 있지 않아 여기선 생략 
    
        
        